In [28]:
import torch
from torchvision import datasets
import torchvision.transforms as transform
from torch.utils.data import DataLoader, WeightedRandomSampler
import torch.optim as optim
import torch.nn as nn
import numpy as np
from ultralytics import YOLO
from torchvision.datasets import ImageFolder
from collections import Counter

In [29]:
class CustomAugmentedDataset(ImageFolder):

    # constructor
    def __init__(self, root, transforms_dict, default_transforms=None):
        super().__init__(root)
        self.transform_dict = transforms_dict
        self.default_transform = default_transforms

    def __getitem__(self, index):
        path, target = self.samples[index]
        sample = self.loader(path)

        class_name = self.classes[target]
        transform = self.transform_dict.get(class_name, self.default_transform)

        if transform:
            sample = transform(sample)
        return sample, target

In [30]:
default_transform = transform.Compose([
    transform.Resize(224),
    transform.CenterCrop(224),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

strong_transform = transform.Compose([
    transform.Resize(224),
    transform.RandomResizedCrop(224),
    transform.RandomHorizontalFlip(),
    transform.RandomRotation(30),
    transform.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transform.ToTensor(),
    transform.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

small_classes = ['red-spot']
transform_dict = {cls: strong_transform for cls in small_classes}

# load dataset beserta transform-nya
train_dataset = CustomAugmentedDataset(root='../../../dataset/Train/', transforms_dict=transform_dict, default_transforms=default_transform)
val_dataset = CustomAugmentedDataset(root='../../../dataset/Valid/', transforms_dict=transform_dict, default_transforms=default_transform)

valid_classes = ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 
                 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
train_class_to_idx = {cls: idx for idx, cls in enumerate(valid_classes) if cls in train_dataset.classes}
train_samples = [(path, train_class_to_idx[train_dataset.classes[label]]) 
                 for path, label in train_dataset.samples 
                 if train_dataset.classes[label] in valid_classes]
val_samples = [(path, train_class_to_idx[val_dataset.classes[label]]) 
               for path, label in val_dataset.samples 
               if val_dataset.classes[label] in valid_classes]

if not train_samples or not val_samples:
    raise ValueError("No samples match the 10 classes. Check dataset subfolders.")

train_dataset.samples = train_samples
train_dataset.classes = valid_classes
train_dataset.class_to_idx = train_class_to_idx
val_dataset.samples = val_samples
val_dataset.classes = valid_classes
val_dataset.class_to_idx = train_class_to_idx

class_counts = np.bincount([label for _, label in train_dataset.samples])
class_weight = 1. / torch.tensor(class_counts, dtype=torch.float)

sample_weights = [class_weight[label] for _, label in train_dataset.samples]
sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

train_dataloader = DataLoader(train_dataset, batch_size=8, sampler=sampler, num_workers=6, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, num_workers=4, shuffle=False)

train_counts = Counter(train_dataset.classes[label] for _, label in train_samples)
val_counts = Counter(val_dataset.classes[label] for _, label in val_samples)
print("Train sample counts per class:")
for cls in valid_classes:
    print(f"{cls}: {train_counts.get(cls, 0)}")
print("Validation sample counts per class:")
for cls in valid_classes:
    print(f"{cls}: {val_counts.get(cls, 0)}")


print(f"Class detected: {train_dataset.classes}")

print("Augmentation summary per class:")
for cls in train_dataset.classes:
    print(f"{cls.ljust(15)} → {'Strong' if cls in transform_dict else 'Default'}")

print(f"Total train batches: {len(train_dataloader)}")
print(f"Total validation batches: {len(val_dataloader)}")

Train sample counts per class:
algal_spot: 1465
brown-blight: 1397
gray-blight: 1220
healthy: 755
helopeltis: 1351
leaf-rust: 1600
red-rust: 417
red-spider-infested: 732
red-spot: 755
white-spot: 233
Validation sample counts per class:
algal_spot: 100
brown-blight: 60
gray-blight: 105
healthy: 45
helopeltis: 155
leaf-rust: 67
red-rust: 5
red-spider-infested: 14
red-spot: 45
white-spot: 32
Class detected: ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot']
Augmentation summary per class:
algal_spot      → Default
brown-blight    → Default
gray-blight     → Default
healthy         → Default
helopeltis      → Default
leaf-rust       → Default
red-rust        → Default
red-spider-infested → Default
red-spot        → Strong
white-spot      → Default
Total train batches: 1241
Total validation batches: 79


In [31]:
# Load YOLOv10 classification model with pre-trained weights
model = YOLO("yolov8n-cls.pt", task="classify")  # Nano model, pre-trained on ImageNet
print(model.task)

classify


In [32]:
num_classes = len(train_dataset.classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('is cuda available?', torch.cuda.is_available())
model = model.to(device)

is cuda available? True


In [33]:
class_weights = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 1.0, 1.0]).to(device)  # Lower weight for red-spider-infested
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [34]:
num_epochs = 15
training_args = {
    'data': '../../../dataset/Train',
    'epochs': num_epochs,
    # 'nc': 10,
    # 'names': ['algal_spot', 'brown-blight', 'gray-blight', 'healthy', 'helopeltis', 'leaf-rust', 'red-rust', 'red-spider-infested', 'red-spot', 'white-spot'],
    'imgsz': 224,
    'batch': 8,
    'device': 0 if torch.cuda.is_available() else 'cpu',
    'workers': 0,
    'project': './runs/train',
    'name': 'yolov8n_cls',
    'exist_ok': True,
    'pretrained': True,
    'optimizer': 'Adam',
    'lr0': 0.0001,
    'patience': 50,
    # Augmentation settings (respecting red-spot's strong augmentation)
    'hsv_h': 0.015,  # Default hue
    'hsv_s': 0.7,    # Default saturation
    'hsv_v': 0.4,    # Default value
    'degrees': 10.0,  # Rotation
    'translate': 0.1, # Translation
    'scale': 0.5,    # Zoom
    'shear': 0.0,
    'flipud': 0.0,   # Vertical flip
    'fliplr': 0.5,   # Horizontal flip
    'mosaic': 0.0,   # Disable mosaic for classification
    'mixup': 0.0,    # Disable mixup
    'task':'classify',
    # 'cls_weight': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 1.0, 1.0]
}

In [35]:
import os
print(os.path.exists("../../../dataset/Valid/"))  # Should print True
print(os.listdir("../../../dataset/Valid/"))  # Should list class folders

True
['leaf-rust', 'brown-blight', 'red-rust', 'white-spot', 'healthy', 'gray-blight', 'helopeltis', 'red-spot', 'algal_spot', 'red-spider-infested']


In [36]:
results = model.train(**training_args)
print(f"result model train YOLO v10: {results}")


Ultralytics 8.3.119 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 SUPER, 3875MiB)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.pt, data=../../../dataset/Train, epochs=15, time=None, patience=50, batch=8, imgsz=224, save=True, save_period=-1, cache=False, device=0, workers=0, project=./runs/train, name=yolov8n_cls, exist_ok=True, pretrained=True, optimizer=Adam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/train... 7937 images, 0 corrupt: 100%|██████████| 7937/7937 [00:00<00:00, 8042.80it/s]


train: New cache created: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 250.6±72.5 MB/s, size: 4.8 KB)


val: Scanning /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/val... 1988 images, 0 corrupt: 100%|██████████| 1988/1988 [00:00<00:00, 7774.62it/s]

val: New cache created: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/val.cache
optimizer: Adam(lr=0.0001, momentum=0.937) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 0 dataloader workers
Logging results to runs/train/yolov8n_cls
Starting training for 15 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...


      15/15     0.424G     0.4909          1        224: 100%|██████████| 993/993 [00:59<00:00, 16.75it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 125/125 [00:08<00:00, 14.65it/s]

                   all      0.872      0.998



15 epochs completed in 0.283 hours.
Optimizer stripped from runs/train/yolov8n_cls/weights/last.pt, 3.0MB
Optimizer stripped from runs/train/yolov8n_cls/weights/best.pt, 3.0MB

Validating runs/train/yolov8n_cls/weights/best.pt...
Ultralytics 8.3.119 🚀 Python-3.12.3 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce GTX 1650 SUPER, 3875MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,447,690 parameters, 0 gradients, 3.3 GFLOPs
WARNING ⚠️ Dataset 'split=train' not found at /home/oz31/code/personal/python/training-skripsi/dataset/Train/train
Found 9518 images in subdirectories. Attempting to split...
Splitting /home/oz31/code/personal/python/training-skripsi/dataset/Train (10 classes, 9925 images) into 80% train, 20% val...
Split complete in /home/oz31/code/personal/python/training-skripsi/dataset/Train_split ✅
train: /home/oz31/code/personal/python/training-skripsi/dataset/Train_split/train... found 9506 images in 10 classes ✅ 
val: /home/oz31/code/personal/python/training-skripsi/dataset/Train_spl

               classes   top1_acc   top5_acc: 100%|██████████| 125/125 [00:08<00:00, 14.48it/s]


                   all      0.876      0.998
Speed: 0.1ms preprocess, 0.4ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/train/yolov8n_cls
result model train YOLO v10: ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7acbe58663f0>
curves: []
curves_results: []
fitness: 0.9373742341995239
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.8762575387954712, 'metrics/accuracy_top5': 0.9984909296035767, 'fitness': 0.9373742341995239}
save_dir: PosixPath('runs/train/yolov8n_cls')
speed: {'preprocess': 0.06103980985806362, 'inference': 0.3656032771615197, 'loss': 0.0003048511073517528, 'postprocess': 0.000496410968519061}
task: 'classify'
top1: 0.8762575387954712
top5: 0.9984909296035767
